<a href="https://colab.research.google.com/github/aknip/Autogen-AutoGPT/blob/main/AutoGen_Content_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Agent


Notes:
- Needs to be tested again due to OpenAI-timeouts, all libs, keys and functions are checked
- Source: https://github.com/JayZeeDesign/microsoft-autogen-experiments/blob/main/content_agent.py




In [26]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [39]:
%%capture --no-stderr
%pip install pyautogen~=0.1.0 docker langchain tiktoken

## Set your API Endpoint


In [32]:
import autogen

config_list = [{'model': 'gpt-4', 'api_key': CREDS['OpenAI']['v1']['credential']}]

# Get API key
load_dotenv()
openai.api_key = CREDS['OpenAI']['v1']['credential']
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v1']['credential']

# Helper functions

In [40]:
from autogen import config_list_from_json
import autogen

import requests
from bs4 import BeautifulSoup

from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
import openai
from dotenv import load_dotenv


def search(query):
    url = "https://google.serper.dev/search"

    payload = json.dumps({
        "q": query
    })
    headers = {
        'X-API-KEY': CREDS['serper_dev']['key']['credential'],
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()


def scrape(url: str):
    # scrape website, and also will summarize the content based on objective if the content is too large
    # objective is the original objective & task that user give to the agent, url is the url of the website to be scraped

    print("Scraping website...")
    # Define the headers for the request
    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json',
    }

    # Define the data to be sent in the request
    data = {
        "url": url
    }

    # Convert Python object to JSON string
    data_json = json.dumps(data)

    # Send the POST request
    response = requests.post(
        "https://chrome.browserless.io/content?token=" + CREDS['browserless_io']['key']['credential'], headers=headers, data=data_json)

    # Check the response status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text()
        print("Scraped Content:", text[:300])
        if len(text) > 8000:
            output = summary(text)
            return output
        else:
            return text
    else:
        print(f"HTTP request failed with status code {response.status_code}")


def summary(content):
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k-0613")
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n"], chunk_size=10000, chunk_overlap=500)
    docs = text_splitter.create_documents([content])
    map_prompt = """
    Write a detailed summary of the following text for a research purpose:
    "{text}"
    SUMMARY:
    """
    map_prompt_template = PromptTemplate(
        template=map_prompt, input_variables=["text"])

    summary_chain = load_summarize_chain(
        llm=llm,
        chain_type='map_reduce',
        map_prompt=map_prompt_template,
        combine_prompt=map_prompt_template,
        verbose=False
    )

    output = summary_chain.run(input_documents=docs,)

    return output


In [ ]:
# Test scrape and summary helpers
#summary1 = scrape("https://de.wikipedia.org/wiki/Theory_of_Constraints")
#print(summary1)

## Construct Agents

In [43]:


def research(query):
    llm_config_researcher = {
        "functions": [
            {
                "name": "search",
                "description": "google search for relevant information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Google search query",
                        }
                    },
                    "required": ["query"],
                },
            },
            {
                "name": "scrape",
                "description": "Scraping website content based on url",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "url": {
                            "type": "string",
                            "description": "Website url to scrape",
                        }
                    },
                    "required": ["url"],
                },
            },
        ],
        "config_list": config_list}

    researcher = autogen.AssistantAgent(
        name="researcher",
        system_message="Research about a given query, collect as many information as possible, and generate detailed research results with loads of technique details with all reference links attached; Add TERMINATE to the end of the research report;",
        llm_config=llm_config_researcher,
    )

    user_proxy = autogen.UserProxyAgent(
        name="User_proxy",
        code_execution_config={"last_n_messages": 2, "work_dir": "coding"},
        is_termination_msg=lambda x: x.get("content", "") and x.get(
            "content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="TERMINATE",
        function_map={
            "search": search,
            "scrape": scrape,
        }
    )

    user_proxy.initiate_chat(researcher, message=query)

    # set the receiver to be researcher, and get a summary of the research report
    user_proxy.stop_reply_at_receive(researcher)
    user_proxy.send(
        "Give me the research report that just generated again, return ONLY the report & reference links", researcher)

    # return the last message the expert received
    return user_proxy.last_message()["content"]


# Define write content function
def write_content(research_material, topic):
    editor = autogen.AssistantAgent(
        name="editor",
        system_message="You are a senior editor of an AI blogger, you will define the structure of a short blog post based on material provided by the researcher, and give it to the writer to write the blog post",
        llm_config={"config_list": config_list},
    )

    writer = autogen.AssistantAgent(
        name="writer",
        system_message="You are a professional AI blogger who is writing a blog post about AI, you will write a short blog post based on the structured provided by the editor, and feedback from reviewer; After 2 rounds of content iteration, add TERMINATE to the end of the message",
        llm_config={"config_list": config_list},
    )

    reviewer = autogen.AssistantAgent(
        name="reviewer",
        system_message="You are a world class hash tech blog content critic, you will review & critic the written blog and provide feedback to writer.After 2 rounds of content iteration, add TERMINATE to the end of the message",
        llm_config={"config_list": config_list},
    )

    user_proxy = autogen.UserProxyAgent(
        name="admin",
        system_message="A human admin. Interact with editor to discuss the structure. Actual writing needs to be approved by this admin.",
        code_execution_config=False,
        is_termination_msg=lambda x: x.get("content", "") and x.get(
            "content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="TERMINATE",
    )

    groupchat = autogen.GroupChat(
        agents=[user_proxy, editor, writer, reviewer],
        messages=[],
        max_round=20)
    manager = autogen.GroupChatManager(groupchat=groupchat)

    user_proxy.initiate_chat(
        manager, message=f"Write a blog about {topic}, here are the material: {research_material}")

    user_proxy.stop_reply_at_receive(manager)
    user_proxy.send(
        "Give me the blog that just generated again, return ONLY the blog, and add TERMINATE in the end of the message", manager)

    # return the last message the expert received
    return user_proxy.last_message()["content"]


# Define content assistant agent
llm_config_content_assistant = {
    "functions": [
        {
            "name": "research",
            "description": "research about a given topic, return the research material including reference links",
            "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The topic to be researched about",
                        }
                    },
                "required": ["query"],
            },
        },
        {
            "name": "write_content",
            "description": "Write content based on the given research material & topic",
            "parameters": {
                    "type": "object",
                    "properties": {
                        "research_material": {
                            "type": "string",
                            "description": "research material of a given topic, including reference links when available",
                        },
                        "topic": {
                            "type": "string",
                            "description": "The topic of the content",
                        }
                    },
                "required": ["research_material", "topic"],
            },
        },
    ],
    "config_list": config_list}

writing_assistant = autogen.AssistantAgent(
    name="writing_assistant",
    system_message="You are a writing assistant, you can use research function to collect latest information about a given topic, and then use write_content function to write a very well written content; Reply TERMINATE when your task is done",
    llm_config=llm_config_content_assistant,
)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    human_input_mode="TERMINATE",
    function_map={
        "write_content": write_content,
        "research": research,
    }
)


## Start Chat

In [44]:
user_proxy.initiate_chat(
    writing_assistant, message="write a blog about autogen multi AI agent framework")

User_proxy (to writing_assistant):

write a blog about autogen multi AI agent framework

--------------------------------------------------------------------------------
writing_assistant (to User_proxy):

***** Suggested function Call: research *****
Arguments: 
{
  "query": "autogen multi AI agent framework"
}
*********************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION research...
User_proxy (to researcher):

autogen multi AI agent framework

--------------------------------------------------------------------------------
researcher (to User_proxy):

***** Suggested function Call: search *****
Arguments: 
{
  "query": "autogen multi AI agent framework"
}
*******************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION search...
User_proxy (t

User_proxy (to researcher):

***** Response from calling function "scrape" *****
The text discusses the Multi-agent Conversation Framework offered by AutoGen, which is a tool that allows for the automation of chat among multiple agents. This framework simplifies the orchestration, automation, and optimization of complex workflows using foundation models. It provides conversable agents that can send and receive messages from other agents, and these agents can be customized to integrate language models, humans, tools, or a combination of them. The framework includes two subclasses of agents: AssistantAgent and UserProxyAgent. The AssistantAgent acts as an AI assistant and can write Python code for a user to execute, while the UserProxyAgent serves as a proxy for humans and can execute code and call functions. The framework supports various conversation patterns, including fully autonomous conversations and human-in-the-loop problem-solving. It also allows for dynamic conversations where 

User_proxy (to researcher):

***** Response from calling function "scrape" *****
The Autogen Multi AI Agent Framework is an open-source software development platform that allows developers to create intelligent systems through multi-agent systems. It consists of agents, environments, communication mechanisms, and learning capabilities, offering flexibility, scalability, interoperability, and extensibility. This framework is revolutionizing intelligent systems in various industries, including robotics, smart cities, finance, and healthcare. It enables the development of advanced robotics systems, intelligent traffic management, trading bots, medical decision support systems, and more. As technology advances, the Autogen Multi AI Agent Framework is expected to power even more innovative AI solutions.

The text also briefly mentions two topics: IT strategy and artificial intelligence (AI). It poses questions about identifying and prioritizing key AI opportunities and challenges for an org

KeyboardInterrupt: ignored